In [2320]:
#Libraries

#Python Libs
import sys
import os
import glob
import traceback
from datetime import datetime
import time
#from geopy import distance


#Data Analysis Libs
import pandas as pd
import numpy as np

In [2321]:
#Functions
def select_input_files(enh_buste_base_path,init_date,fin_date,suffix):
        selected_files = []
        all_files = glob.glob(os.path.join(enh_buste_base_path,"*"))

        for file_ in all_files:
                try:
                        file_date = pd.to_datetime(file_.split('/')[-1],format=('%Y_%m_%d' + suffix  + '.csv'))
                        if (file_date >= init_date) and (file_date <= fin_date):
                                selected_files.append((file_,file_date))
                except:
                        continue

        return sorted(selected_files)

def get_gtfs_path(query_date):
    INTERMEDIATE_OTP_DATE = pd.to_datetime("2019-04-05", format="%Y-%m-%d")
    router_id = ''

    if (query_date <= INTERMEDIATE_OTP_DATE):
        return 'campina-gtfs-2019'
    else:
        return 'campina-gtfs-2017'

In [2322]:
## Main

In [2323]:
otp_suggestions_folderpath = 'data/output/output_merged_otp_bulma/march'
#otp_suggestions_filepath = 'data/output/2019_02_02_bus_trips_otp_itineraries.csv'
bus_trips_folderpath = 'data/input/bus_trips/march'
gtfs_base_folderpath = 'data/input'
output_folderpath = 'data/output'

In [2324]:
# for file in os.listdir(otp_suggestions_folderpath):
#     otp_suggestions_filepath = otp_suggestions_folderpath + "/" + file
#     print otp_suggestions_filepath

In [2325]:
#gtfs_base_folderpath + os.sep + get_gtfs_path(file_date) + os.sep
otp_suggestions_filepath = 'data/output/output_otp/march/2019_03_29_bus_trips_otp_itineraries.csv'

In [2326]:
file_date_str = otp_suggestions_filepath.split('/')[-1].split('_bus_trips_')[0]
file_date = pd.to_datetime(file_date_str,format='%Y_%m_%d')
print "Processing File:", otp_suggestions_filepath

Processing File: data/output/output_otp/march/2019_03_29_bus_trips_otp_itineraries.csv


In [2327]:
otp_suggestions_filepath

'data/output/output_otp/march/2019_03_29_bus_trips_otp_itineraries.csv'

In [2328]:
print pd.to_datetime(1549023680000/1000, unit='s')
print pd.to_datetime(1549095223000/1000, unit='s')

2019-02-01 12:21:20
2019-02-02 08:13:43


In [2329]:
    # Extracting itinerary part name for later use
    itinerary_part_name = otp_suggestions_filepath.split('/')[-1].split('_')[5]
    # Read OTP Suggestions
    otp_suggestions_raw = pd.read_csv(otp_suggestions_filepath, parse_dates=['date','otp_start_time','otp_end_time'])

#     if len(otp_suggestions_raw) == 0:
#         print "Zero OTP suggestions found."
#         print "Skipping next steps..."
#         exit(0)


In [2330]:
otp_suggestions_raw.head(10)

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
0,2019-03-29,491551.0,1,1,2019-03-30 08:18:45,2019-03-30 08:18:46,WALK,NaN,NaN,NaN,0.016667
1,2019-03-29,491551.0,1,2,2019-03-30 08:18:47,2019-03-30 08:48:31,BUS,944,491551.0,386521.0,29.733333
2,2019-03-29,491551.0,1,3,2019-03-30 08:56:33,2019-03-30 09:20:43,BUS,220,386521.0,386259.0,24.166667
3,2019-03-29,491551.0,1,4,2019-03-30 09:20:44,2019-03-30 09:23:13,WALK,NaN,NaN,NaN,2.483333
4,2019-03-29,491551.0,2,1,2019-03-30 08:53:45,2019-03-30 08:53:46,WALK,NaN,NaN,NaN,0.016667
5,2019-03-29,491551.0,2,2,2019-03-30 08:53:47,2019-03-30 09:19:58,BUS,944,491551.0,385744.0,26.183333
6,2019-03-29,491551.0,2,3,2019-03-30 09:19:58,2019-03-30 09:23:39,WALK,NaN,NaN,NaN,3.683333
7,2019-03-29,491551.0,2,4,2019-03-30 09:30:54,2019-03-30 09:41:04,BUS,220,386637.0,386259.0,10.166667
8,2019-03-29,491551.0,2,5,2019-03-30 09:41:05,2019-03-30 09:43:34,WALK,NaN,NaN,NaN,2.483333
9,2019-03-29,491551.0,3,1,2019-03-30 09:11:45,2019-03-30 09:11:46,WALK,NaN,NaN,NaN,0.016667


In [2331]:
def prepare_otp_data(otp_data):
        #Fixing prefix
        otp_data.columns = otp_data.columns.str.replace('otp_','')
        otp_data = otp_data.add_prefix('otp_')
        
        #Fixing Timezone difference - when needed
        otp_data['otp_start_time'] = otp_data['otp_start_time'] - pd.Timedelta('10800 s')
        otp_data['otp_end_time'] = otp_data['otp_end_time'] - pd.Timedelta('10800 s')
        
        #Adjusting route format to have 3 numbers
        otp_data['otp_route'] = otp_data['otp_route'].astype(str)
        otp_data['otp_route'] = np.where(otp_data['otp_mode'] == 'BUS',
                            otp_data['otp_route'].astype(str).str.replace("\.0",'').str.zfill(3),
                            otp_data['otp_route'])

        return otp_data

In [2332]:
    # Prepare OTP data for analysis
    otp_suggestions = prepare_otp_data(otp_suggestions_raw)    
    
    # Read stops data
    stops_filepath = gtfs_base_folderpath + os.sep + get_gtfs_path(file_date) + os.sep + 'stops.txt'
    stops_df = pd.read_csv(stops_filepath)
    
    # Adding Parent Stop data to OTP Suggestions TODO
    stops_parent_stations = stops_df[['stop_id','parent_station']]
    otp_suggestions = otp_suggestions.merge(stops_parent_stations.add_prefix('from_'),
                                                left_on='otp_from_stop_id',
                                                right_on='from_stop_id',
                                                how='left') \
                                        .merge(stops_parent_stations.add_prefix('to_'),
                                                left_on='otp_to_stop_id',
                                                right_on='to_stop_id',
                                                how='left') \
                                        .drop(['from_stop_id','to_stop_id'], axis=1) \
                                        .rename(index=str, columns={'from_parent_station':'otp_from_parent_station',
                                                                    'to_parent_station':'otp_to_parent_station'})
        
    otp_suggestions_bus_legs = otp_suggestions[otp_suggestions['otp_mode'] == 'BUS']
    otp_suggestions_walk_legs = otp_suggestions[otp_suggestions['otp_mode'] == 'WALK']

In [2333]:
#otp_suggestions.drop_duplicates(subset=['otp_leg_id','otp_end_time','otp_mode','otp_duration_mins'],inplace=True)
#otp_suggestions.duplicated(subset=['otp_leg_id','otp_end_time','otp_mode','otp_duration_mins'])
otp_suggestions.head(10)

,otp_date,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_start_time,otp_end_time,otp_mode,otp_route,otp_from_stop_id,otp_to_stop_id,otp_duration_mins,otp_from_parent_station,otp_to_parent_station
0,2019-03-29,491551.0,1,1,2019-03-30 05:18:45,2019-03-30 05:18:46,WALK,nan,NaN,NaN,0.016667,NaN,NaN
1,2019-03-29,491551.0,1,2,2019-03-30 05:18:47,2019-03-30 05:48:31,BUS,944,491551.0,386521.0,29.733333,NaN,NaN
2,2019-03-29,491551.0,1,3,2019-03-30 05:56:33,2019-03-30 06:20:43,BUS,220,386521.0,386259.0,24.166667,NaN,NaN
3,2019-03-29,491551.0,1,4,2019-03-30 06:20:44,2019-03-30 06:23:13,WALK,nan,NaN,NaN,2.483333,NaN,NaN
4,2019-03-29,491551.0,2,1,2019-03-30 05:53:45,2019-03-30 05:53:46,WALK,nan,NaN,NaN,0.016667,NaN,NaN
5,2019-03-29,491551.0,2,2,2019-03-30 05:53:47,2019-03-30 06:19:58,BUS,944,491551.0,385744.0,26.183333,NaN,NaN
6,2019-03-29,491551.0,2,3,2019-03-30 06:19:58,2019-03-30 06:23:39,WALK,nan,NaN,NaN,3.683333,NaN,NaN
7,2019-03-29,491551.0,2,4,2019-03-30 06:30:54,2019-03-30 06:41:04,BUS,220,386637.0,386259.0,10.166667,NaN,NaN
8,2019-03-29,491551.0,2,5,2019-03-30 06:41:05,2019-03-30 06:43:34,WALK,nan,NaN,NaN,2.483333,NaN,NaN
9,2019-03-29,491551.0,3,1,2019-03-30 06:11:45,2019-03-30 06:11:46,WALK,nan,NaN,NaN,0.016667,NaN,NaN


## Read and Prepare Bus Trip Data

In [2334]:
bus_trips_folderpath + os.sep + file_date_str + '_bus_trips.csv'

'data/input/bus_trips/march/2019_03_29_bus_trips.csv'

In [ ]:
bus_trips_filepath = bus_trips_folderpath + os.sep + file_date_str + '_bus_trips.csv'
bus_trips = pd.read_csv(bus_trips_filepath, dtype={'route': object},parse_dates=['gps_datetime']) \
                                        .sort_values(['route','busCode','tripNum','gps_datetime']) \
                                        .assign(route = lambda x: x['route'].astype(str).str.replace("\.0",'').str.zfill(3))  \
                                        .drop_duplicates()

In [ ]:
bus_trips.head(10)

In [ ]:
#bus_trips[(bus_trips.route == "944") & (bus_trips.stopPointId == 491551.0)].head()
#bus_trips['gps_datetime'] = pd.to_datetime(bus_trips['gps_datetime'], format='%Y_%m_%d %H:%M:%S')

In [ ]:
#trezentos = bus_trips[(bus_trips.route == "333")]
#output = trezentos.to_csv("data/output/trezentos.csv",index=False)


In [ ]:
bus_trips_clean = bus_trips.filter(['route','busCode','tripNum','stopPointId','gps_datetime'])

In [ ]:
bus_trips_clean.head()

In [ ]:
#bus_trips_clean[bus_trips_clean.route == "944"].head()

In [ ]:
bus_trips_clean.dtypes

In [ ]:
otp_suggestions_bus_legs.dtypes

## Identify Possible Matches between OTP Itineraries and Bus Trips Observed Data

In [ ]:
#bus_trips_clean.gps_datetime = bus_trips_clean.drop(bus_trips_clean.gps_datetime[bus_trips_clean.gps_datetime == "-"].index, inplace=True)

bus_trips_clean.drop(bus_trips_clean.loc[bus_trips_clean['gps_datetime']=='-'].index, inplace=True)


#bus_trips_clean = bus_trips_clean[bus_trips_clean.gps_datetime == "-"]
bus_trips_clean.dtypes
#bus_trips_clean.gps_datetime = pd.to_datetime(bus_trips_clean.gps_datetime)
#pd.to_datetime('01-02-2019 06:01:01')
bus_trips_clean.head()

In [ ]:
bus_trips_clean['gps_datetime'] = pd.to_datetime(bus_trips_clean['gps_datetime'], format='%d-%m-%Y %H:%M:%S')
bus_trips_clean.head()

In [ ]:
scheduled_itin_observed_o = otp_suggestions_bus_legs.merge(bus_trips_clean.add_prefix('bt_'),
                                left_on=['otp_route','otp_from_stop_id'],
                                right_on=['bt_route','bt_stopPointId'],
                                how='inner') \
                                .drop(['bt_route','bt_stopPointId'], axis=1) \
                                .rename(index=str, columns={'bt_gps_datetime':'bt_start_time',
                                                            'bt_tripNum':'bt_trip_num',
                                                            'bt_busCode':'bt_bus_code'}) \
                                .assign(sched_obs_start_timediff = 
                                        lambda x: np.absolute(pd.to_datetime(x['bt_start_time']) - x['otp_start_time']))

In [ ]:
#bus_trips_clean[(bus_trips_clean.route == "944") & (bus_trips_clean.stopPointId == 491551.0)].sort_values(by=['gps_datetime']).head()

In [ ]:
scheduled_itin_observed_o.head(10)

In [ ]:
#scheduled_itin_observed_o[(scheduled_itin_observed_o.otp_route == "944") & (scheduled_itin_observed_o.otp_start_time == '2019-05-13 14:05:47')][['otp_itinerary_id',"otp_start_time","otp_route","otp_from_stop_id",
                                                                           #"otp_from_stop_id", "bt_start_time", "sched_obs_start_timediff"]].sort_values(by=['sched_obs_start_timediff']).head()
#scheduled_itin_observed_o[(scheduled_itin_observed_o.otp_itinerary_id == 112)]

In [ ]:
scheduled_itin_observed_o_1 = scheduled_itin_observed_o[(scheduled_itin_observed_o.otp_mode == "BUS") & (scheduled_itin_observed_o.sched_obs_start_timediff >= pd.Timedelta('0s'))  ]#& (scheduled_itin_observed_o.sched_obs_start_timediff < pd.Timedelta('1.5h'))]
scheduled_itin_observed_o_1

In [ ]:
scheduled_itin_observed_od = scheduled_itin_observed_o.merge(bus_trips_clean.add_prefix('bt_'),
                                left_on=['otp_route','bt_bus_code','bt_trip_num','otp_to_stop_id'],
                                right_on=['bt_route','bt_busCode','bt_tripNum','bt_stopPointId'],
                                how='inner') \
                                .drop(['bt_route','bt_stopPointId'], axis=1) \
                                .rename(index=str, columns={'bt_gps_datetime':'bt_end_time'}) \
                                .assign(sched_obs_end_timediff = 
                                        lambda x: np.absolute(x['bt_end_time'] - x['otp_end_time'])) \
                                .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','sched_obs_start_timediff','sched_obs_end_timediff'])

In [ ]:
scheduled_itin_observed_od.head()

In [ ]:
scheduled_itin_observed_od['bt_duration_mins'] = (scheduled_itin_observed_od['bt_end_time'] - scheduled_itin_observed_od['bt_start_time'])/pd.Timedelta(minutes=1)
scheduled_itin_observed_od = scheduled_itin_observed_od[scheduled_itin_observed_od['bt_duration_mins'] > 0]

In [ ]:
scheduled_itin_observed_od

In [ ]:
scheduled_itin_observed_od_full = pd.concat([scheduled_itin_observed_od,otp_suggestions_walk_legs], sort=False)
scheduled_itin_observed_od_full

In [ ]:
otp_suggestions_walk_legs

In [ ]:
#scheduled_itin_observed_od_full[scheduled_itin_observed_od_full.otp_itinerary_id == 296]

In [ ]:
#bus_trips_clean[(bus_trips['route'] == '944') & (bus_trips['stopPointId'] == 491551)].sort_values(['gps_datetime'])

In [ ]:
scheduled_itin_observed_od_full_clean = scheduled_itin_observed_od_full \
                            .filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','otp_mode','otp_route',
                                     'bt_bus_code','bt_trip_num','otp_from_stop_id','otp_start_time',
                                     'bt_start_time','sched_obs_start_timediff','otp_to_stop_id',
                                     'otp_end_time','bt_end_time','sched_obs_end_timediff','otp_duration_mins','minimun_obs_start_time','bt_duration_mins']) \
                            .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])

In [ ]:
scheduled_itin_observed_od_full.head() \
                            .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])

In [ ]:
# coisa = scheduled_itin_observed_od_full_clean[(scheduled_itin_observed_od_full_clean.otp_route == "944") & (scheduled_itin_observed_od_full_clean.otp_start_time == '2019-05-13 15:05:47')][['otp_itinerary_id',"otp_start_time","otp_route","otp_from_stop_id",
#                                                                            "otp_from_stop_id", "bt_start_time", "sched_obs_start_timediff"]].sort_values(by=['sched_obs_start_timediff'])

# coisa

In [ ]:
scheduled_itin_observed_od_full_clean = scheduled_itin_observed_od_full \
                            .filter(['otp_user_trip_id','otp_itinerary_id','otp_leg_id','otp_mode','otp_route',
                                     'bt_bus_code','bt_trip_num','otp_from_stop_id','otp_start_time',
                                     'bt_start_time','sched_obs_start_timediff','otp_to_stop_id',
                                     'otp_end_time','bt_end_time','sched_obs_end_timediff','otp_duration_mins','minimun_obs_start_time','bt_duration_mins']) \
                            .sort_values(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])


In [ ]:
scheduled_itin_observed_od_full_clean

In [ ]:
scheduled_itin_observed_od_full_clean \
                        .groupby(['otp_itinerary_id', 'otp_leg_id']) \
                        .apply(lambda x: x.sort_values(["sched_obs_start_timediff"]))


In [ ]:
#scheduled_itin_observed_od_full_clean[(scheduled_itin_observed_od_full_clean.otp_route == "944") & (scheduled_itin_observed_od_full_clean.otp_start_time == '2019-05-13 14:05:47') & (scheduled_itin_observed_od_full_clean.otp_itinerary_id == 230)][['otp_itinerary_id','otp_leg_id',"otp_start_time","otp_route","otp_from_stop_id",
                                                                           #"otp_from_stop_id", "bt_start_time", "sched_obs_start_timediff"]].sort_values(by=['sched_obs_start_timediff']).head()

In [ ]:
scheduled_itin_observed_od_full_clean = scheduled_itin_observed_od_full_clean.drop_duplicates(subset=['otp_itinerary_id','otp_leg_id'])

In [ ]:
scheduled_itin_observed_od_full_clean.head()

In [ ]:
#scheduled_itin_observed_od_full_clean[["otp_itinerary_id","otp_leg_id","otp_mode","otp_route","bt_start_time","bt_end_time"]]

In [ ]:
# Filtering out itineraries which lost bus legs along the processing


curr_matched_itins_num_legs = scheduled_itin_observed_od_full_clean.groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                    .agg({'otp_leg_id': lambda x: len(np.unique(x))}) \
                                    .reset_index() \
                                    .rename(index=str, columns={'otp_leg_id':'num_legs'})

In [ ]:
original_suggested_itins_num_legs = otp_suggestions.groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                    .agg({'otp_leg_id': lambda x: len(x)}) \
                                    .reset_index() \
                                    .rename(index=str, columns={'otp_leg_id':'num_legs'})

In [ ]:
original_suggested_itins_num_legs.dtypes

In [ ]:
complete_matched_itins = original_suggested_itins_num_legs

In [ ]:
complete_matched_itins.head()

In [ ]:
all_complete_vehicle_legs_options = scheduled_itin_observed_od_full_clean.merge(complete_matched_itins.drop('num_legs', axis=1), how='inner')

In [ ]:
all_complete_vehicle_legs_options.drop_duplicates(keep='first',subset=['otp_mode','otp_route','bt_bus_code','bt_trip_num', 'otp_from_stop_id','otp_start_time','bt_start_time','sched_obs_start_timediff','otp_to_stop_id','otp_end_time','bt_end_time','sched_obs_end_timediff','otp_duration_mins'])


In [ ]:
output = all_complete_vehicle_legs_options.to_csv("data/output/output_merged_otp_bulma/march/output_" + file_date_str  + ".csv",index=False)

In [2134]:
def is_new_itinerary(prev_trip_id,curr_trip_id,prev_itin_id,curr_itin_id):
    return ((prev_trip_id != curr_trip_id) | (prev_itin_id != curr_itin_id))

def choose_leg_matches(leg_matches_groups):
        colnames = leg_matches_groups.obj.columns.values
        chosen_leg_matches = pd.DataFrame(columns = colnames)
        prev_trip_id = -1
        prev_itin_id = -1
        prev_leg_mode = ""
        prev_leg_end_time = pd.NaT
        num_groups_not_survived = 0
        new_itinerary = False

        for name, group in leg_matches_groups:
            
                #print
                #print "Name:", name
                #print "Group:"
                #print group
                #print
                
                curr_trip_id = group['otp_user_trip_id'].iloc[0]
                curr_itin_id = group['otp_itinerary_id'].iloc[0]
                curr_leg_id = group['otp_leg_id'].iloc[0]
                curr_leg_mode = group['otp_mode'].iloc[0]
                curr_leg_route = group['otp_route'].iloc[0]
                #print curr_leg_route
                
                new_itinerary = is_new_itinerary(prev_trip_id,curr_trip_id,prev_itin_id,curr_itin_id)
                if new_itinerary:
                    prev_leg_end_time = group['otp_start_time'].dt.floor('d').iloc[0]

                #if (prev_group_id == ()):
                #        prev_leg_end_time = group['bt_start_time'].dt.floor('d')[0]

                #print
                #print "Previous itinerary id:", prev_itin_id
                #print "Previous leg mode:", prev_leg_mode
                #print "Previous leg end time:", prev_leg_end_time
                #print "Current leg id:", curr_leg_id
                #print "Current leg mode:", curr_leg_mode
                #print
                #print "Original Group"
                #print group.filter(['otp_start_time','bt_start_time','bt_end_time'])
                
                if (curr_leg_mode == 'WALK'):
                    #print "Walking duration:", filtered_group['otp_duration_mins']
                    filtered_group = group.reset_index()
                    if new_itinerary: #first leg is a WALK leg
                        filtered_group.loc[0,'bt_end_time'] = prev_leg_end_time
                    else:
                        filtered_group.loc[0,'bt_start_time'] = prev_leg_end_time
                        filtered_group.loc[0,'bt_end_time'] = prev_leg_end_time + \
                            pd.Timedelta(minutes=np.rint(filtered_group['otp_duration_mins'].iloc[0]))
                    #print "Filtered Group"
                    #print filtered_group
                else:
                    filtered_group = group[group['bt_start_time'] > prev_leg_end_time]
                
                #print
                #print "Filtered Group"
                #print filtered_group.filter(['otp_start_time','bt_start_time','bt_end_time'])

                if (len(filtered_group) == 0):
                        #print "Group did not survive! =("
                        #print
                        #print "Previous itinerary id:", prev_itin_id
                        #print "Previous leg mode:", prev_leg_mode
                        #print "Previous leg end time:", prev_leg_end_time
                        #print "Current leg id:", curr_leg_id
                        #print "Current leg mode:", curr_leg_mode
                        #print
                        #print "Original Group"
                        #print group#.filter(['otp_start_time','bt_start_time','bt_end_time'])
                        num_groups_not_survived += 1
                        continue

                chosen_leg_match = filtered_group.sort_values('bt_start_time').iloc[0]
                #print chosen_leg_match
                #if (curr_leg_id == 2):
#                 print curr_leg_id
#                 print curr_leg_mode
#                 print prev_leg_mode
                
                if ((curr_leg_id == 2) & 
                    ((curr_leg_mode == 'BUS') & (prev_leg_mode == 'WALK'))):
                        #Update previous walk start/end_times
                        #print
                        #print "Chosen Leg Matches"
                        #print chosen_leg_matches.iloc[-1]
                        #print
                        chosen_leg_matches.iloc[-1,chosen_leg_matches.columns.get_loc('bt_start_time')] = chosen_leg_match['bt_start_time'] - \
                            pd.Timedelta(minutes=np.rint(chosen_leg_matches.iloc[-1].otp_duration_mins))
                        chosen_leg_matches.iloc[-1,chosen_leg_matches.columns.get_loc('bt_end_time')] = chosen_leg_match['bt_start_time']
                #print "Chosen Leg"
                #print chosen_leg_match

                chosen_leg_matches = chosen_leg_matches.append(chosen_leg_match)

                #Update variables
                #prev_group_id = name
                prev_trip_id = curr_trip_id
                prev_itin_id = curr_itin_id
                prev_leg_mode = curr_leg_mode
                prev_leg_end_time = chosen_leg_match['bt_end_time']

        #print "Number of groups which did not survive:", num_groups_not_survived
        return chosen_leg_matches.filter(colnames)

In [2135]:
legs_groups = all_complete_vehicle_legs_options.groupby(['otp_user_trip_id','otp_itinerary_id','otp_leg_id'])

In [2136]:
legs_groups.obj.columns.values

array(['otp_user_trip_id', 'otp_itinerary_id', 'otp_leg_id', 'otp_mode',
       'otp_route', 'bt_bus_code', 'bt_trip_num', 'otp_from_stop_id',
       'otp_start_time', 'bt_start_time', 'sched_obs_start_timediff',
       'otp_to_stop_id', 'otp_end_time', 'bt_end_time',
       'sched_obs_end_timediff', 'otp_duration_mins', 'bt_duration_mins'],
      dtype=object)

In [2137]:
# for key, item in legs_groups:
#     print(legs_groups.get_group(key), "\n\n")

In [2138]:
# Choose best actual leg matches (based on feasibility and start time)
import time

start = time.time()
#chosen_legs = choose_leg_matches(sample_itinerary_options.groupby(['otp_user_trip_id','otp_itinerary_id','otp_leg_id']))
feasible_legs = choose_leg_matches(legs_groups)
end = time.time()

print "Execution time in s:", (end-start)


Execution time in s: 1.24706482887


In [2139]:
feasible_legs

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,sched_obs_start_timediff,otp_to_stop_id,otp_end_time,bt_end_time,sched_obs_end_timediff,otp_duration_mins,bt_duration_mins
0,491551.0,1,1,WALK,nan,NaN,NaN,NaN,2019-03-09 18:18:45,2019-03-09 17:12:21,NaT,NaN,2019-03-09 18:18:46,2019-03-09 17:12:21,NaT,0.016667,NaN
1,491551.0,1,2,BUS,944,1064,11,491551.0,2019-03-09 18:18:47,2019-03-09 17:12:21,0 days 01:06:26,497505.0,2019-03-09 18:33:21,2019-03-09 17:44:12,0 days 00:49:09,14.566667,31.850000
2,491551.0,1,3,BUS,220,1091,14,386521.0,2019-03-09 19:01:54,2019-03-09 19:40:32,0 days 00:38:38,386259.0,2019-03-09 19:26:04,2019-03-09 19:57:55,0 days 00:31:51,24.166667,17.383333
0,491551.0,1,5,WALK,nan,NaN,NaN,NaN,2019-03-09 19:09:10,2019-03-09 19:57:55,NaT,NaN,2019-03-09 19:09:22,2019-03-09 19:57:55,NaT,0.200000,NaN
0,491551.0,2,1,WALK,nan,NaN,NaN,NaN,2019-03-09 18:53:45,2019-03-09 17:12:21,NaT,NaN,2019-03-09 18:53:46,2019-03-09 17:12:21,NaT,0.016667,NaN
6,491551.0,2,2,BUS,944,1064,11,491551.0,2019-03-09 18:53:47,2019-03-09 17:12:21,0 days 01:41:26,497505.0,2019-03-09 19:08:21,2019-03-09 17:44:12,0 days 01:24:09,14.566667,31.850000
7,491551.0,2,3,BUS,220,1091,14,386521.0,2019-03-09 19:34:54,2019-03-09 19:40:32,0 days 00:05:38,386259.0,2019-03-09 19:59:04,2019-03-09 19:57:55,0 days 00:01:09,24.166667,17.383333
0,491551.0,2,5,WALK,nan,NaN,NaN,NaN,2019-03-09 19:49:10,2019-03-09 19:57:55,NaT,NaN,2019-03-09 19:49:22,2019-03-09 19:57:55,NaT,0.200000,NaN
0,491551.0,3,1,WALK,nan,NaN,NaN,NaN,2019-03-09 19:20:03,NaT,NaT,NaN,2019-03-09 19:30:48,2019-03-09 00:00:00,NaT,10.750000,NaN
11,491551.0,3,3,BUS,111,3024,11,386521.0,2019-03-09 20:08:36,2019-03-09 19:30:03,0 days 00:38:33,386087.0,2019-03-09 20:17:13,2019-03-09 19:41:06,0 days 00:36:07,8.616667,11.050000


In [2140]:
if len(feasible_legs) == 0:
    print "No matches left after matching and selecting feasible bus legs."
    print "Skipping next steps..."
    exit(0)

# Filtering out itineraries which lost bus legs after feasible legs choice processing
feasible_itins_num_legs = feasible_legs.groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                    .agg({'otp_leg_id': lambda x: len(x)}) \
                                    .reset_index() \
                                    .rename(index=str, columns={'otp_leg_id':'num_legs'})
feasible_itins_num_legs.head(20)


,otp_user_trip_id,otp_itinerary_id,num_legs
0,491551.0,1,4
1,491551.0,2,4
2,491551.0,3,4
3,491551.0,4,3
4,491551.0,5,4
5,491551.0,6,4
6,491551.0,7,5
7,491551.0,8,2
8,491551.0,9,2
9,491551.0,10,2


In [2141]:
feasible_itins_num_legs.head()

,otp_user_trip_id,otp_itinerary_id,num_legs
0,491551.0,1,4
1,491551.0,2,4
2,491551.0,3,4
3,491551.0,4,3
4,491551.0,5,4


In [2142]:
original_suggested_itins_num_legs.head()

,otp_user_trip_id,otp_itinerary_id,num_legs
0,491551.0,1,29
1,491551.0,2,31
2,491551.0,3,31
3,491551.0,4,31
4,491551.0,5,32


In [2143]:
feasible_complete_itins = feasible_itins_num_legs.merge(original_suggested_itins_num_legs,how='inner')

feasible_complete_itins.head(20)

,otp_user_trip_id,otp_itinerary_id,num_legs


In [2144]:
feasible_complete_itins_legs = feasible_legs.merge(feasible_complete_itins.drop('num_legs', axis=1),how='inner')

feasible_complete_itins_legs.head(10)

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,sched_obs_start_timediff,otp_to_stop_id,otp_end_time,bt_end_time,sched_obs_end_timediff,otp_duration_mins,bt_duration_mins


In [2145]:
def add_stops_data_to_legs(itineraries_legs,stops_locs):
    itineraries_legs_stops = itineraries_legs.merge(stops_locs, left_on='otp_from_stop_id', right_on='stop_id', how='left') \
                                                                                .drop('stop_id', axis=1) \
                                                                                .rename(index=str, columns={'stop_lat':'from_stop_lat','stop_lon':'from_stop_lon'}) \
                                                                                .merge(stops_locations, left_on='otp_to_stop_id', right_on='stop_id', how='left') \
                                                                                .drop('stop_id', axis=1) \
                                                                                .rename(index=str, columns={'stop_lat':'to_stop_lat','stop_lon':'to_stop_lon'}) 
    return itineraries_legs_stops

In [2146]:
stops_locations = stops_df[['stop_id','stop_lat','stop_lon']]
itineraries_legs = add_stops_data_to_legs(feasible_complete_itins_legs,stops_locations)

itineraries_legs.head()

,otp_user_trip_id,otp_itinerary_id,otp_leg_id,otp_mode,otp_route,bt_bus_code,bt_trip_num,otp_from_stop_id,otp_start_time,bt_start_time,...,otp_to_stop_id,otp_end_time,bt_end_time,sched_obs_end_timediff,otp_duration_mins,bt_duration_mins,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon


In [2147]:
"data/output/output/output_merged_otp_bulma" + file_date_str  + ".csv"

'data/output/output/output_merged_otp_bulma2019_03_09.csv'

In [2148]:
feasible_complete_itins_legs.head()
output = feasible_complete_itins_legs.to_csv("data/output/output/output_merged_otp_bulma" + file_date_str  + ".csv",index=False)
#feasible_complete_itins_legs[["otp_itinerary_id","otp_leg_id","otp_mode","otp_route","bt_start_time","bt_end_time"]]

IOError: [Errno 2] No such file or directory: 'data/output/output/output_merged_otp_bulma2019_03_09.csv'

In [ ]:
teste = feasible_complete_itins_legs.drop_duplicates(keep='first',subset=['otp_leg_id','otp_mode','otp_route','bt_bus_code','bt_trip_num', 'otp_from_stop_id','otp_start_time','bt_start_time',
                'sched_obs_start_timediff','otp_to_stop_id','otp_end_time','bt_end_time','sched_obs_end_timediff'])
#feasible_complete_itins_legs_drop_duplicates = choose_leg_matches_drop_duplicates(feasible_complete_itins_legs)
#output = teste.to_csv("data/output/output.csv",index=False)
teste


In [ ]:
#output2 = teste.to_csv("data/output/output/output_merged_otp_bulma/2019_01_01.csv",index=False)

In [ ]:
teste = teste[~teste.duplicated(['otp_leg_id','otp_mode','otp_route','bt_bus_code','bt_trip_num', 'otp_from_stop_id','otp_start_time','bt_start_time',
                'sched_obs_start_timediff','otp_to_stop_id','otp_end_time','bt_end_time','sched_obs_end_timediff'])
              .groupby(teste['otp_itinerary_id']).transform('any')]
teste
#output = teste.to_csv("data/output/output_2019_02_10.csv",index=False)


In [ ]:
grouped = teste.groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                    .agg({'otp_leg_id': lambda x: len(x) < 5}).reset_index()


In [ ]:
grouped2 = teste.groupby(['otp_user_trip_id','otp_itinerary_id']).filter(lambda g: len(g.otp_leg_id) == 5)
grouped2



In [ ]:
grouped3 = grouped2.groupby(['otp_user_trip_id','otp_itinerary_id']) \
                                    .agg({'otp_leg_id': lambda x: len(x) < 5})
grouped3